In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time

# CBAM (Convolutional Block Attention Module) Layer
class CBAM(layers.Layer):
    def __init__(self):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention()
        self.spatial_attention = SpatialAttention()
    
    def call(self, inputs):
        channel_att = self.channel_attention(inputs)
        spatial_att = self.spatial_attention(inputs)
        return tf.multiply(inputs, tf.expand_dims(tf.expand_dims(channel_att, axis=1), axis=1)) * spatial_att

# Channel Attention Layer
class ChannelAttention(layers.Layer):
    def __init__(self):
        super(ChannelAttention, self).__init__()
        self.global_avgpool = layers.GlobalAveragePooling2D()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        avg_pool = self.global_avgpool(inputs)
        dense1_out = self.dense1(avg_pool)
        attention = self.dense2(dense1_out)
        return attention

# Linear Attention Layer
class LinearAttention(layers.Layer):
    def __init__(self):
        super(LinearAttention, self).__init__()
        self.dense = layers.Dense(1)
    
    def call(self, inputs):
        return tf.nn.sigmoid(self.dense(inputs))

# NonLinear Attention Layer
class NonLinearAttention(layers.Layer):
    def __init__(self):
        super(NonLinearAttention, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        dense1_out = self.dense1(inputs)
        attention = self.dense2(dense1_out)
        return attention

def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'],debugflag=False):
    if debugflag:
        model = tf.keras.Sequential([
            layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(2, 1)),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')
        ])
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        return model
        
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        layer_type = layer_config[0]

        if layer_type == 'dense':
            neurons, activation = layer_config[1], layer_config[2]
            if len(layer_config) > 3 and layer_config[3] == 'input':
                input_shape = layer_config[4]
                model.add(layers.Dense(neurons, activation=activation, input_shape=(input_shape,)))
            else:
                model.add(layers.Dense(neurons, activation=activation))
        elif layer_type == 'dropout':
            rate = layer_config[1]
            model.add(layers.Dropout(rate))
        elif layer_type == 'activation':
            activation = layer_config[1]
            model.add(layers.Activation(activation))
        elif layer_type == 'conv2d':
            filters, kernel_size, strides, activation = layer_config[1], layer_config[2], layer_config[3], layer_config[4]
            model.add(layers.Conv2D(filters, kernel_size=kernel_size, strides=strides, activation=activation))
        elif layer_type == 'maxpool2d':
            pool_size = layer_config[1]
            model.add(layers.MaxPool2D(pool_size=pool_size))
        elif layer_type == 'rnn':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.SimpleRNN(units, return_sequences=return_sequences))
        elif layer_type == 'lstm':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.LSTM(units, return_sequences=return_sequences))
        elif layer_type == 'gru':
            units, return_sequences = layer_config[1], layer_config[2]
            model.add(layers.GRU(units, return_sequences=return_sequences))
        elif layer_type == 'attention':
            model.add(layers.MultiHeadAttention(num_heads=layer_config[1], key_dim=layer_config[2]))
        elif layer_type == 'cbam':
            model.add(CBAM())
        elif layer_type == 'channel_attention':
            model.add(ChannelAttention())
        elif layer_type == 'linear_attention':
            model.add(LinearAttention())
        elif layer_type == 'nonlinear_attention':
            model.add(NonLinearAttention())

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model


def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50,ea=False,eamonitor="val_loss"):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor=eamonitor, patience=10, min_delta=0.001, verbose=1)


    start_time = time.time()
    if ea:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2",threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test,threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    #threshold = 0.5  # Define the threshold
    interpredictions=predictions.flatten()
    correct_predictions = np.sum(np.abs(true_labels - interpredictions) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, interpredictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, interpredictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), interpredictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }


In [3]:
# Load dataset
dataset_name = "addition_dataset_2"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 94342992.0000 - mae: 5051.7275 - val_loss: 0.0600 - val_mae: 0.2041
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0618 - mae: 0.2072 - val_loss: 0.0600 - val_mae: 0.2054
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0635 - mae: 0.2083 - val_loss: 0.0598 - val_mae: 0.2038
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0611 - mae: 0.2059 - val_loss: 0.0596 - val_mae: 0.2037
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0610 - mae: 0.2057 - val_loss: 0.0595 - val_mae: 0.2022
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0610 - mae: 0.2058 - val_loss: 0.0601 - val_mae: 0.2018
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0603 - mae: 0.2041 - val_loss: 0.0597 - val_mae: 0.2075
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0592 - mae: 0.2023 - val_loss: 0.0586 - val_mae: 0.2047
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - los

In [4]:



# Load dataset
dataset_name = "substraction_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)


Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2987537.5000 - mae: 660.8404 - val_loss: 0.0050 - val_mae: 0.0590
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0051 - mae: 0.0595 - val_loss: 0.0050 - val_mae: 0.0586
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0051 - mae: 0.0595 - val_loss: 0.0050 - val_mae: 0.0584
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0051 - mae: 0.0595 - val_loss: 0.0049 - val_mae: 0.0585
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0050 - mae: 0.0590 - val_loss: 0.0048 - val_mae: 0.0584
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0050 - mae: 0.0590 - val_loss: 0.0048 - val_mae: 0.0581
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0049 - mae: 0.0582 - val_loss: 0.0048 - val_mae: 0.0587
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0048 - mae: 0.0580 - val_loss: 0.0049 - val_mae: 0.0603
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss:

In [8]:
# Load dataset
dataset_name = "doubling_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=50,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/50


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 131119760.0000 - mae: 6609.3330 - val_loss: 0.2151 - val_mae: 0.3859
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.4426 - mae: 0.4245 - val_loss: 0.2144 - val_mae: 0.3870
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2171 - mae: 0.3874 - val_loss: 0.2141 - val_mae: 0.3877
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2172 - mae: 0.3876 - val_loss: 0.2132 - val_mae: 0.3861
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2218 - mae: 0.3922 - val_loss: 0.2126 - val_mae: 0.3861
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2155 - mae: 0.3861 - val_loss: 0.2115 - val_mae: 0.3851
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2186 - mae: 0.3895 - val_loss: 0.2101 - val_mae: 0.3841
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.2137 - mae: 0.3851 - val_loss: 0.2106 - val_mae: 0.3790
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - lo

In [12]:
# Load dataset
dataset_name = "multiplication_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adamW', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=500,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 32253505733394432.0000 - mae: 134046632.0000 - val_loss: 29191067349811200.0000 - val_mae: 126391144.0000
Epoch 2/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27229261580468224.0000 - mae: 120118152.0000 - val_loss: 17562322063065088.0000 - val_mae: 92637344.0000
Epoch 3/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 14447863035592704.0000 - mae: 82255104.0000 - val_loss: 5116885523759104.0000 - val_mae: 49062100.0000
Epoch 4/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3957078192291840.0000 - mae: 44889772.0000 - val_loss: 2081823127502848.0000 - val_mae: 39058884.0000
Epoch 5/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2045416300347392.0000 - mae: 39065008.0000 - val_loss: 1997908995997696.0000 - val_mae: 39666948.0000
Epoch 6/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1983064515280896.0000 - mae: 39328348.0000 - val_loss: 1999276808863744.0000 - val_mae: 39779592.0000
Epoch 7/500
500/500 ━━━━━━

In [14]:
# Load dataset
dataset_name = "simple_sin_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=500,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 218329.9531 - mae: 163.9324 - val_loss: 0.5046 - val_mae: 0.6410
Epoch 2/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5153 - mae: 0.6417 - val_loss: 0.5707 - val_mae: 0.6596
Epoch 3/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5147 - mae: 0.6408 - val_loss: 0.5277 - val_mae: 0.6483
Epoch 4/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5250 - mae: 0.6456 - val_loss: 0.5159 - val_mae: 0.6447
Epoch 5/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5186 - mae: 0.6394 - val_loss: 0.5048 - val_mae: 0.6411
Epoch 6/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5394 - mae: 0.6487 - val_loss: 0.5360 - val_mae: 0.6509
Epoch 7/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5430 - mae: 0.6504 - val_loss: 0.5046 - val_mae: 0.6411
Epoch 8/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5775 - mae: 0.6621 - val_loss: 0.5254 - val_mae: 0.6464
Epoch 9/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step 

In [16]:
# Load dataset
dataset_name = "simple_cos_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=500,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 86313.7031 - mae: 100.7560 - val_loss: 0.5012 - val_mae: 0.6361
Epoch 2/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5171 - mae: 0.6436 - val_loss: 0.5046 - val_mae: 0.6373
Epoch 3/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5311 - mae: 0.6470 - val_loss: 0.5289 - val_mae: 0.6434
Epoch 4/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5450 - mae: 0.6543 - val_loss: 0.6412 - val_mae: 0.6819
Epoch 5/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5803 - mae: 0.6623 - val_loss: 0.5670 - val_mae: 0.6552
Epoch 6/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5707 - mae: 0.6600 - val_loss: 0.4993 - val_mae: 0.6355
Epoch 7/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5863 - mae: 0.6680 - val_loss: 0.4980 - val_mae: 0.6346
Epoch 8/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.5919 - mae: 0.6680 - val_loss: 0.6783 - val_mae: 0.6923
Epoch 9/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step -

In [17]:
# Load dataset
dataset_name = "exponent_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv',nrows=16)
d_set['result']=d_set['result'].astype('int')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=500,ea=True)

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 912ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 546036612727832576.0000 - mae: 516242720.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 546036612727832576.0000 - mae: 516242784.0000 - val_loss: 148205629029220352.0000 - val_mae: 192491328.0000
Epoch 7/500
1/1

In [19]:
# Load dataset
dataset_name = "natural_log_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Reshape features to include timestep dimension
# Assuming each row in the CSV file represents a timestep
# Reshape to (number_of_samples, timesteps, number_of_features)
timesteps = 1  # Assuming each row represents a timestep
dset_features_reshaped = dset_features.reshape(-1, timesteps, dset_features.shape[1])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers

model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(dset_features.shape[1], 1)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,model_name='CNN',dataset_name=dataset_name,total_epoch=500,ea=True,eamonitor='loss')

# Test the model
test_metrics = test_model(model, X_test, y_test,model_name='CNN',dataset_name=dataset_name)

Epoch 1/500


c:\Users\akars\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 139082.8750 - mae: 136.2335 - val_loss: 13.9708 - val_mae: 3.2220
Epoch 2/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 14.3451 - mae: 3.2550 - val_loss: 13.7366 - val_mae: 3.1814
Epoch 3/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.9550 - mae: 3.2037 - val_loss: 13.6191 - val_mae: 3.2115
Epoch 4/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.7324 - mae: 3.1782 - val_loss: 13.4927 - val_mae: 3.1097
Epoch 5/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.3130 - mae: 3.1385 - val_loss: 15.3596 - val_mae: 3.3478
Epoch 6/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.2506 - mae: 3.1374 - val_loss: 11.9869 - val_mae: 2.9378
Epoch 7/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 12.5704 - mae: 3.0571 - val_loss: 11.9171 - val_mae: 3.0314
Epoch 8/500
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 11.9668 - mae: 2.9940 - val_loss: 15.0732 - val_mae: 3.3564
Epoch 9/500
500/500 ━━━━━━━━━━━━━━━━━━